# Tổng quan pipeline PV (dành cho người không chuyên)

Notebook này mô tả quy trình xử lý dữ liệu và huấn luyện mô hình cho hệ thống PV (năng lượng mặt trời) theo cách thật dễ hiểu. Nội dung gồm: input, output, thuật toán, cách đánh giá kết quả, và ứng dụng thực tế.

## 1. Import thư viện cần thiết
Dòng code dưới đây nhập những thư viện Python cơ bản để xử lý dữ liệu, huấn luyện mô hình và vẽ hình.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve

# Hiển thị đồ họa đẹp
%matplotlib inline

## 2. Định nghĩa cấu trúc pipeline (các bước chính)
Ở mức cao, pipeline gồm các bước: đọc dữ liệu -> tiền xử lý -> phụ thuộc tính (feature) -> huấn luyện mô hình -> đánh giá -> lưu kết quả. Dưới đây là một cấu trúc hàm đơn giản để gọi tuần tự các bước.

In [ ]:
def read_data(path=None):
    # Nếu có đường dẫn CSV thì đọc, nếu không tạo dữ liệu mẫu
    if path:
        return pd.read_csv(path)
    # dữ liệu mẫu nhỏ để demo
    rng = np.random.RandomState(42)
    n = 1000
    df = pd.DataFrame({
        'facility_code': rng.choice(['F1','F2','F3'], size=n),
        'date_hour': pd.date_range('2025-01-01', periods=n, freq='H'),
        'energy_mwh': rng.gamma(2.0, 1.0, size=n),
        'temperature_2m': rng.normal(25, 5, size=n),
        'cloud_cover': rng.rand(n),
        'pm2_5': rng.normal(10, 3, size=n)
    })
    return df

def pipeline_run(df):
    df2 = preprocess(df)
    X, y = make_features_labels(df2)
    model, metrics = train_and_eval(X, y)
    return model, metrics, df2

## 3. Chuẩn bị input: đọc dữ liệu mẫu và kiểm tra
Ví dụ: nếu chưa có dữ liệu, chúng ta tạo dữ liệu mẫu (như ở trên). Sau khi có dữ liệu, hãy kiểm tra vài dòng đầu và kiểu cột.

In [ ]:
# Đọc dữ liệu mẫu
df = read_data()
print('Kích thước dữ liệu:', df.shape)
df.head()

## 4. Tiền xử lý dữ liệu: làm sạch và chuẩn hoá
Tiền xử lý đơn giản: xử lý giá trị thiếu, tạo nhãn (label), và chuẩn hoá một vài cột số.

In [ ]:
def preprocess(df):
    df = df.copy()
    # loại bỏ dòng không có energy
    df = df[df['energy_mwh'].notnull()]
    # ví dụ: tạo nhãn 'energy_high' nếu energy >= 60th percentile
    thresh = df['energy_mwh'].quantile(0.6)
    df['energy_high'] = (df['energy_mwh'] >= thresh).astype(int)
    # chọn vài cột số và điền giá trị thiếu bằng trung bình
    for c in ['temperature_2m','cloud_cover','pm2_5']:
        if c in df.columns:
            df[c] = df[c].fillna(df[c].mean())
    return df

def make_features_labels(df):
    # Ví dụ đơn giản: dùng 3 feature để demo
    features = ['temperature_2m','cloud_cover','pm2_5']
    X = df[features].values
    y = df['energy_high'].values
    return X, y

# Demo tiền xử lý
df2 = preprocess(df)
df2[['energy_mwh','energy_high','temperature_2m','cloud_cover','pm2_5']].head()

## 5. Cài đặt thuật toán chính (mô tả dễ hiểu + mã mẫu)
Chúng ta dùng Logistic Regression (một thuật toán phân loại đơn giản). Nói ngắn: mô hình học cách phân biệt hai nhóm (nhiều năng lượng / ít năng lượng) từ các đặc trưng.

In [ ]:
def train_and_eval(X, y):
    # chia train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # khởi tạo mô hình và huấn luyện
    model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train)
    # dự đoán và tính metric đơn giản
    y_pred = model.predict(X_test)
    y_score = model.predict_proba(X_test)[:,1]
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, zero_division=0),
        'recall': recall_score(y_test, y_pred, zero_division=0),
        'f1': f1_score(y_test, y_pred, zero_division=0),
        'auc': roc_auc_score(y_test, y_score) if len(np.unique(y_test))>1 else None
    }
    return model, metrics

# Demo huấn luyện nhanh
X, y = make_features_labels(df2)
model, metrics = train_and_eval(X, y)
metrics

## 6. Chạy pipeline và thu output: chạy từng bước và lưu kết quả
Ví dụ chạy toàn bộ pipeline và lưu model/metrics vào file (CSV / pickle).

In [ ]:
# Chạy pipeline mẫu
model, metrics, df_processed = pipeline_run(df)
print('Metrics (tóm tắt):')
print(metrics)
# Lưu metrics ra file JSON/CSV (ví dụ)
metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv('output_metrics_sample.csv', index=False)
print('Đã lưu metrics vào output_metrics_sample.csv')

## 7. Đánh giá kết quả: metric đơn giản và kiểm tra chất lượng
Giải thích ngắn gọn các metric đã dùng và cách hiểu kết quả:

- Accuracy: tỉ lệ dự đoán đúng trên tổng số.
- Precision: khi mô hình dự đoán dương (năng lượng cao), tỉ lệ đó đúng bao nhiêu.
- Recall: tỉ lệ thực tế dương được mô hình tìm ra.
- F1: trung bình điều hoà precision và recall.
- AUC: đo khả năng phân biệt lớp dương và âm theo toàn bộ ngưỡng; càng gần 1 càng tốt.

In [ ]:
# Hiển thị các metric đã lưu
print(metrics)

## 8. Trực quan hoá kết quả: biểu đồ và bảng
Ví dụ: vẽ ROC curve, biểu đồ phân phối năng lượng, và ma trận nhầm lẫn (confusion matrix).

In [ ]:
# Ví dụ vẽ histogram năng lượng
plt.figure(figsize=(8,4))
sns.histplot(df['energy_mwh'], bins=30)
plt.title('Phân phối năng lượng (MWh)')
plt.xlabel('energy_mwh')
plt.show()

# ROC curve (nếu có AUC)
try:
    # tái tạo test split để plot
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    y_score = model.predict_proba(X_test)[:,1]
    fpr, tpr, _ = roc_curve(y_test, y_score)
    plt.figure(figsize=(6,4))
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc_score(y_test, y_score):.3f}')
    plt.plot([0,1],[0,1], '--', color='grey')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()
except Exception as e:
    print('Không thể vẽ ROC (thiếu dữ liệu hoặc chỉ một lớp):', e)

## 9. Ví dụ chạy toàn bộ: script demo và cách mở trong VS Code
- Bạn có thể mở file notebook này trong VS Code bằng extension Python / Jupyter.
- Nhấn 'Run Cell' hoặc chạy toàn bộ bằng 'Run All' để thực hiện từng bước.

Lệnh chạy script training thực tế (trong dự án) nếu dùng Docker + Spark:

In [ ]:
# Lệnh ví dụ dùng docker-compose (chạy trong terminal, không phải trong notebook)
print('Docker run example:')
print(

)

## 10. Gợi ý ứng dụng và bước tiếp theo (dành cho người vận hành)
- Ứng dụng: giám sát hiệu suất nhà máy theo giờ, phát hiện bất thường, cảnh báo khi sản lượng giảm.
- Bước tiếp theo: theo dõi metric theo thời gian trên MLflow, kiểm tra phân tích theo facility, và cải thiện dữ liệu đầu vào (thêm feature thời tiết chi tiết, dữ liệu thiết bị).
- Nếu gặp lỗi khi upload model/artifact lên MinIO: kiểm tra quyền (policy), access key và bucket đã cấu hình đúng trong MLflow.

---
Cảm ơn — nếu bạn muốn, tôi có thể:
1) Chuyển notebook này thành tiếng Anh,
2) Thêm phần hướng dẫn chi tiết để kết nối MLflow <-> MinIO,
3) Thêm cell tự động tải artifacts từ MLflow để hiển thị đồ họa trong notebook.